# Sonar - Decentralized Model Training Simulation (local)

DISCLAIMER: This is a proof-of-concept implementation. It does not represent a remotely product ready implementation or follow proper conventions for security, convenience, or scalability. It is part of a broader proof-of-concept demonstrating the vision of the OpenMined project, its major moving parts, and how they might work together.


# Getting Started: Installation

##### Step 1: install IPFS

- https://ipfs.io/docs/install/

##### Step 2: Turn on IPFS Daemon
Execute on command line:
> ipfs daemon

##### Step 3: Install Ethereum testrpc

- https://github.com/ethereumjs/testrpc

##### Step 4: Turn on testrpc with 1000 initialized accounts (each with some money)
Execute on command line:
> testrpc -a 1000

##### Step 5: install openmined/sonar and all dependencies
##### Step 6: execute the following code

# The Simulation: Diabetes Prediction

In this example, a diabetes research center (Cure Diabetes Inc) wants to train a model to try to predict the progression of diabetes based on several indicators. They have collected a small sample (42 patients) of data but it's not enough to train a model. So, they intend to offer up a bounty of $5,000 to the OpenMined commmunity to train a high quality model.

As it turns out, there are 400 diabetics in the network who are candidates for the model (are collecting the relevant fields). In this simulation, we're going to faciliate the training of Cure Diabetes Inc incentivizing these 400 anonymous contributors to train the model using the Ethereum blockchain.

Note, in this simulation we're only going to use the sonar and syft packages (and everything is going to be deployed locally on a test blockchain). Future simulations will incorporate mine and capsule for greater anonymity and automation.

### Setting up the Experiment

In [1]:
import numpy as np
import phe as paillier
from sonar.contracts import ModelMine
from syft.he.paillier import KeyPair
from syft.nn.linear import LinearClassifier
import numpy as np
from sklearn.datasets import load_diabetes

In [2]:
# for the purpose of the simulation, we're going to split our dataset up amongst
# the relevant simulated users

diabetes = load_diabetes()
y = diabetes.target
X = diabetes.data

cure_diabetes_validation_dataset = (X[0:42],y[0:42])
anonymous_diabetes_users = (X[42:],y[42:])

In [3]:
# we're also going to initialize the model trainer smart contract, which in the
# real world would already be on the blockchain (managing other contracts) before
# the simulation begins

mine = ModelMine() # openmined account is default[2]

NameError: name 'contract' is not defined

In [21]:
# we're going to set aside 400 accounts for our 400 patients
patient_addresses = mine.web3.eth.accounts[100:500]
print(patient_addresses[0:10])

# we're going to set aside 1 account for Cure Diabetes Inc
cure_diabetes_inc = mine.web3.eth.accounts[501]

['0x496c578d7adb72fecbf82106d85c9e56cdd6dd61', '0x407ae516b28272ecbed1171567bc2ec962c1dcf7', '0x72b0be2a2b59043b62a3714f1b5232c2cb8a7c89', '0xe8e3e3e9d2501774849683d0dca6e8ff434f728a', '0xbbe477fcb70f00d5e0bb74b823f63eb3515fcb68', '0x9d2c093305ab544d102b1d665063451a5b2f583d', '0x932fe90e14a6adcf724409be97eee4fd060183a6', '0x3588232d499a2e8c0b56e13680ccd6607d8814a6', '0x8bf1482155292b55997413cfec3e2bbb166130ba', '0x1008e610d7ea16c8bbd67ad17eb7e18449d169e6']


In [22]:
# Let's go ahead and pair each data point with each patient's address so that we know we don't get them confused
anonymous_diabetics = list(zip(patient_addresses,anonymous_diabetes_users[0],anonymous_diabetes_users[1]))

### Step 1: Cure Diabetes Inc Initializes a Model and Provides a Bounty

In [23]:
pubkey,prikey = KeyPair().generate(n_length=1024)
diabetes_model = LinearClassifier(desc="DiabetesModel",n_inputs=10,n_labels=1).encrypt(pubkey)

In [14]:
model_id = mine.submit_model(diabetes_model)

/Users/amberedmundson/anaconda/lib/python3.6/site-packages/eth_abi/abi.py:91: DeprecationWarning: Automatic inference of hex encoded data has been deprecated. Please adjust your code to ensure that the data argument for `decode_single` is a byte string
  "Automatic inference of hex encoded data has been deprecated. "


In [17]:
mine.submit_gradient(model_id,diabetes_model)

/Users/amberedmundson/anaconda/lib/python3.6/site-packages/eth_abi/abi.py:91: DeprecationWarning: Automatic inference of hex encoded data has been deprecated. Please adjust your code to ensure that the data argument for `decode_single` is a byte string
  "Automatic inference of hex encoded data has been deprecated. "


1

In [8]:
model_client = mine[model_id]

for iter in range(10):
    print(iter)
    model_client.learn([0,0,0,1],np.array([0,1]).astype('float64'))

/Users/amberedmundson/anaconda/lib/python3.6/site-packages/eth_abi/abi.py:91: DeprecationWarning: Automatic inference of hex encoded data has been deprecated. Please adjust your code to ensure that the data argument for `decode_single` is a byte string
  "Automatic inference of hex encoded data has been deprecated. "


0
1
2
3
4
5
6
7
8
9


In [9]:
updated_id = mine.submit_model(model_client)

/Users/amberedmundson/anaconda/lib/python3.6/site-packages/eth_abi/abi.py:91: DeprecationWarning: Automatic inference of hex encoded data has been deprecated. Please adjust your code to ensure that the data argument for `decode_single` is a byte string
  "Automatic inference of hex encoded data has been deprecated. "


In [10]:
# download newly trained model, decrypt it and predict
model_server = mine[updated_id].decrypt(prikey)
pred = model_server.forward(input=[0,0,0,1])
pred

/Users/amberedmundson/anaconda/lib/python3.6/site-packages/eth_abi/abi.py:91: DeprecationWarning: Automatic inference of hex encoded data has been deprecated. Please adjust your code to ensure that the data argument for `decode_single` is a byte string
  "Automatic inference of hex encoded data has been deprecated. "


array([ 0.        ,  0.99902344])